In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, pyarrow as pa
import os, re, chardet, xlrd
from IPython.display import display, HTML, Markdown
from openpyxl import load_workbook

##### Toggle pandas limit

In [2]:
# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

### Batch Import
The below will get the sheet names of the master Excel file as a list, and create a second list of dataframes generated from each sheet.  Then, it will pickle (.pkl) each dataframe with a filename based on the corresponding sheet, and save them to be quickly accessed later.

In [ ]:
def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

sheetnames = get_sheetnames_xlsx('Vision_Data/Vision Data Master Copy.xlsx')

dfs = []

## This section takes a long time to run, uncomment when needed.
for sheet in sheetnames:
    dfs.append(pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name=sheet))
    
df_names = ['AR', 'CL', 'LD', 'LedgerAP', 'LedgerAR', 'LedgerEX', 
            'LedgerMisc', 'Opportunity', 'OpportunityCustomTabFields', 'PR', 
            'Projects_BillingProjections', 'Projects_ProjectSetup']

In [ ]:
# # single import for minor correction
# df_AR = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='AR')
# df_CL = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='CL')
# df_EMProjectAssoc = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='EMProjectAssoc')
# df_LD = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='LD')
# df_LedgerAP = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='LedgerAP')
# df_LedgerAR = pd.read_excel("Vision_Data/Vision Data Master Copy.xlsx", sheet_name="LedgerAR")
# df_LedgerEX = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='LedgerEX')
# df_LedgerMisc = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='LedgerMisc')
# df_Opportunity = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='Opportunity')
# df_PR = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='PR')
# df_Projects_BillingProjections = pd.read_excel('Vision_data/Vision Data Master Copy.xlsx', sheet_name='Projects_BillingProjections')
# df_Projects_ProjectSetup = pd.read_excel('Vision_data/Vision Data Master Copy.xlsx', sheet_name='Projects_ProjectSetup')

In [ ]:
for i, df in enumerate(dfs):
    columns = df.columns
    print(f"{df_names[i]} has columns: {columns}")

In [ ]:
# some basic data cleaning before pickling

In [22]:
df_AR = dfs[0]
# DueDate & LinkCompany are null, and RetainageDate only has 531 values, & is unlikely to be useful
# df_AR.drop("DueDate", axis=1, inplace=True)
# df_AR.drop("LinkCompany", axis=1, inplace=True)
# df_AR.drop("RetainageDate", axis=1, inplace=True)
df_AR.dropna(axis=1, how='all', inplace=True)

In [23]:
# possibly do some basic data cleaning before pickling
df_CL = dfs[1]

In [5]:
## DF_CL take 2, from SQL csv (12/19/23)
df_CL = pd.read_csv('Vision_Data/SQL/CL.csv', index_col=0)
df_CL.dropna(axis=1, how='all', inplace=True)
df_CL.columns

Index(['ClientID', 'Client', 'Name', 'Status', 'ExportInd', 'WebSite',
       'CurrentStatus', 'CustomCurrencyCode', 'CreateUser', 'CreateDate',
       'ModUser', 'ModDate', 'ClientInd', 'VendorInd', 'LinkedVendor',
       'PriorWork', 'Recommend', 'DisadvBusiness',
       'DisabledVetOwnedSmallBusiness', 'HBCU', 'MinorityBusiness',
       'SmallBusiness', 'VetOwnedSmallBusiness', 'WomanOwned', 'AlaskaNative',
       'SpecialtyType', 'Employees', 'AnnualRevenue', 'GovernmentAgency',
       'Competitor', 'EightA', 'Hubzone', 'Incumbent', 'AjeraSync'],
      dtype='object')

In [10]:
# df_CL.drop(axis=1, labels=["ParentID", "ParentLevel1", "ParentLevel2", "ParentLevel3",
#                            "CustomCurrencyCode", "Type", "Specialty"], inplace=True)

## these aren't present in the new version, I'm not sure why not

In [11]:
df_CL[df_CL['Name'].isna()]
## also no longer an issue
# df_CL[4660:4670]

,ClientID,Client,Name,Status,ExportInd,WebSite,CurrentStatus,CustomCurrencyCode,CreateUser,CreateDate,...,AlaskaNative,SpecialtyType,Employees,AnnualRevenue,GovernmentAgency,Competitor,EightA,Hubzone,Incumbent,AjeraSync


In [13]:
# df_CL.to_pickle('Vision_Data/working_data/CL.pkl')
## and that's it.  Back to original code from here on

In [24]:
df_CL.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_CL.drop(axis=1, labels=["ParentID", "ParentLevel1", "ParentLevel2", "ParentLevel3",
                           "CustomCurrencyCode", "Type", "Specialty"], inplace=True)

In [26]:
df_CL[df_CL['Name'].isna()]

,ClientID,Client,Name,Status,ExportInd,WebSite,Memo,CurrentStatus,CreateUser,CreateDate,...,AlaskaNative,SpecialtyType,Employees,AnnualRevenue,GovernmentAgency,Competitor,EightA,Hubzone,Incumbent,AjeraSync
4667,"T: #f0f""",Prospect,NaN,2013-04-04 12:26:26,TCOCHERL,2013-04-09 12:31:36,Y,N,N,N,...,NaN,0,N,N,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_CL.drop(4667, inplace=True)

In [28]:
# df_CL[4660:4670]

# index now has a gap. This will close it
df_CL.index = range(len(df_CL))

In [29]:
# df_EMProjectAssoc = dfs[???]
# this one went away I guess?  Troubleshoot later if we actually end up needing it

In [30]:
# df_EMProjectAssoc.dropna(axis=1, how='all', inplace=True)

In [31]:
# df_EMProjectAssoc.isna().sum()

In [32]:
df_LD = dfs[2]
df_LD.dropna(axis=1, how='all', inplace=True)

In [ ]:
df

In [33]:
df_LD.drop(axis=1, labels=["TimekeeperEndDate","Payrate", "PayOvtPct", "PaySpecialOvtPct", 
                           "SpecialOvtAmt", "SpecialOvtAmtBillingCurrency", "SpecialOvtAmtEmployeeCurrency",
                           "SpecialOvtAmtProjectCurrency", "SpecialOvtRateBillingCurrency",
                           "SpecialOvtRateEmployeeCurrency", "SpecialOvtRateProjectCurrency",
                           "ProjectCost", "SpecialOvtHrs","SpecialOvtAmt", "SpecialOvtPct", 
                           "SpecialOvtRate", "Pool", "BillingExchangeInfo", "TLProcessed"], inplace=True) 
                            # basically null
# df_LD.drop(axis=1, labels=["DebitLedgerMiscPKey", "CreditLedgerMiscPKey"], inplace=True) 
                            # Don't know what these two are, probably not needed
df_LD.drop(axis=1, labels=["PostSeq", "EmType", "SuppressBill", "SelPeriod", "SelPostSeq",
                           "SelOvtPeriod", "SelOvtPostSeq", "SelPeriod",
                           "SelPostSeq", "CostRateTableUsed"], inplace=True) 
                            # potentially useful but taking them out for now
df_LD.drop(axis=1, labels=["TransType"], inplace=True) 
                            # only has values LA (> 100 entries) and TES (the other 1M +) don't know what they mean, probably not useful.
df_LD.drop(axis=1, labels=["RegAmtBillingCurrency", "RegAmtEmployeeCurrency",
                           "RegAmtProjectCurrency", "OvtAmtBillingCurrency", 
                           "OvtAmtEmployeeCurrency", "OvtAmtProjectCurrency",
                           "RateBillingCurrency", "RateEmployeeCurrency",
                           "RateProjectCurrency", "OvtRateBillingCurrency", 
                           "OvtRateEmployeeCurrency", "OvtRateProjectCurrency",
                           "RealizationAmountBillingCurrency", "RealizationAmountEmployeeCurrency",
                           "RealizationAmountProjectCurrency"], inplace=True)
                            # Unlikely to be needed

In [34]:
# Check null counts
null_counts = df_LD.isna().sum()
# null_counts[0:25]

In [35]:
df_LD.columns

Index(['Period', 'PKey', 'WBS1', 'WBS2', 'WBS3', 'LaborCode', 'Employee',
       'TransDate', 'Name', 'RegHrs', 'OvtHrs', 'RegAmt', 'OvtAmt', 'BillExt',
       'Rate', 'OvtPct', 'OvtRate', 'Category', 'EmOrg', 'PrOrg', 'ChargeType',
       'RateType', 'DebitLedgerMiscPKey', 'CreditLedgerMiscPKey', 'BillStatus',
       'Comment', 'BilledWBS1', 'BilledWBS2', 'BilledWBS3', 'BilledInvoice',
       'BilledPeriod', 'XferWBS1', 'XferWBS2', 'XferWBS3', 'XferLaborCode',
       'WrittenOffPeriod', 'ProjectExchangeInfo', 'ExchangeInfo',
       'XferCategory', 'NonBill', 'TransferredPeriod',
       'TransferredBillStatus'],
      dtype='object')

In [36]:
df_LD.drop(axis=1, labels=["DebitLedgerMiscPKey","CreditLedgerMiscPKey",
                           "ProjectExchangeInfo", "ExchangeInfo",
                          "TransferredPeriod", "TransferredBillStatus"], inplace=True) 

In [ ]:
df_LedgerAP = dfs[3]
df_LedgerAP.dropna(axis=1, how='all', inplace=True)

In [ ]:
nullseries = df_LedgerAP.isnull().sum()
print(nullseries[nullseries > 0])

In [ ]:
df_LedgerAP.shape

In [ ]:
df_LedgerAR = dfs[4]
df_LedgerAR.dropna(axis=1, how='all', inplace=True)

In [ ]:
# Check null counts
null_counts = df_LedgerAR.isna().sum()
null_counts

In [ ]:
null_counts[0:30]

In [ ]:
# B - To be billed, W - To be written off, H - Held, F - Final billed, 
# X - Written off, T - Transferred, I - Inserted, M - Modified, 
# D - To be deleted, N - Not billable, O - Deleted, R - Partial hold/Released.
df_LedgerAR["TransferredPeriod"].value_counts()

In [ ]:
df_LedgerAR.drop(axis=1, labels=[], inplace=True)

In [ ]:
df_LedgerAR.drop(axis=1, labels=["PostSeq", "Account", "CBAmount", "CBAmountTaxCurrency",
                                "CreditMemoRefNo", "SuppressBill", "AutoEntry",
                                "AutoEntryAmount", "SkipGL", "BankCode", "PartialPayment", 
                                 "Line", "Discount", "UnitBillExt", "UnitBillingRate", 
                                 "UnitCostRate", "UnitCostRateBillingCurrency", "UnitQuantity",
                                 "TaxBasis", "TaxBasisFunctionalCurrency", "TaxBasisTaxCurrency",
                                 "TaxCBBasis", "TaxCBBasisFunctionalCurrency", "ExchangeInfo",
                                 "TaxCode", "XferAccount", "TransferredBillStatus",
                                 "BillingExchangeInfo", "ProjectExchangeInfo", "SubType",
                                 "TaxCBBasisTaxCurrency", "WrittenOffPeriod", 
                                 "AmountBillingCurrency", "AmountSourceCurrency",
                                 "AmountProjectCurrency", "AmountTaxCurrency",
                                 "UnitCostRateBillingCurrency", "TaxCBBasis",
                                 "TaxCBBasisFunctionalCurrency", "TaxCBBasisTaxCurrency",
                                 "DiscountFunctionalCurrency", "RealizationAmountBillingCurrency",
                                 "RealizationAmountEmployeeCurrency", "RealizationAmountProjectCurrency",
                                 "OriginalAmountSourceCurrency", "InProcessAccount",
                                 "InProcessAccountCleared", "EKOriginalLine", "DiaryNo",
                                 "TransferredPeriod"], inplace=True)

In [ ]:
df_LedgerAR.shape

In [ ]:
df_LedgerEX = dfs[5]
df_LedgerEX.dropna(axis=1, how='all', inplace=True)

In [ ]:
nullseries = df_LedgerEX.isnull().sum()
print(nullseries[nullseries > 0])

In [ ]:
df_LedgerMisc = dfs[6]
# df_LedgerMisc.dropna(axis=1, how='all', inplace=True)

In [ ]:
nullseries = df_LedgerMisc.isnull().sum()
print(nullseries[nullseries > 0])

In [ ]:
df_LedgerMisc_TEST = pd.read_excel('Vision_Data/Vision Data Master Copy.xlsx', sheet_name='LedgerMisc')

In [ ]:
df_LedgerMisc_TEST['AutoEntryAccount'].isna().sum() / len(df_LedgerMisc_TEST['AutoEntryAccount'])

In [ ]:
df_LedgerMisc.drop('AutoEntryOrg', axis=1, inplace=True)
df_LedgerMisc.drop('AutoEntryAccount', axis=1, inplace=True)
# already dropped somewhere else? keeps erroring, even though these should be in the df

In [ ]:
df_Opportunity = dfs[7]

In [ ]:
df_Opportunity.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_Opportunity.isna().sum()

In [ ]:
df_Opportunity["PRProposalWBS1"].value_counts()

In [ ]:
df_Opportunity.drop(axis=1, labels=["ProposalManager", "OurRole", "NAICS", "CompetitionType", 
                                    "ContractTypeGovCon", "Duration", "AwardType", "ClosedNotes",
                                    "ClosedReason", "CustomCurrencyCode", "SolicitationNum",
                                    "PublicNoticeDate", "Source", "Opportunity"], inplace=True)

In [ ]:
df_Opportunity.shape

In [ ]:
df_OpportunityCustomTabFields = dfs[8]

In [ ]:
df_OpportunityCustomTabFields.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_OpportunityCustomTabFields.isna().sum()

In [ ]:
# df_PR = dfs[9] # old version

In [44]:
## updating DF_PR from SQL, 12-19-23

# dtype doesn't handle datetime, so I need this
parse_dates = ['EndDate', 'ActCompletionDate']

df_PR = pd.read_csv('Vision_Data/SQL/PR.csv',
                    dtype={'BillWBS1': str, 'BillWBS2': str, 'BillWBS3': str, 'Address2': str, 'Address3': str,
                           'Zip': str, 'County': str, 'BidDate': str, 'ClientAlias': str,  'FirmCostComment': str,
                           'TotalCostComment': str, 'OpportunityID': str, 'Phone': str, 'Fax': str, 'EMail': str,
                           'ProposalWBS1': str, 'ProfServicesComplDate': str, 'RevUpsetWBS2': str, 'RevUpsetWBS3': str,
                           'PhoneFormat': str, 'FaxFormat': str},
                    parse_dates=parse_dates, 
                    index_col=0)

df_PR[['BillWBS1', 'BillWBS2', 'BillWBS3', 'Address2', 'Address3', 'Zip', 'County', 'BidDate',
       'ClientAlias', 'FirmCostComment', 'TotalCostComment', 'OpportunityID', 'Phone', 'Fax',
       'EMail', 'ProposalWBS1', 'ProfServicesComplDate', 'RevUpsetWBS2', 'RevUpsetWBS3',
       'PhoneFormat', 'FaxFormat']].fillna('', inplace=True)

C:\Users\jhoward\AppData\Local\Temp\ipykernel_28556\1341259585.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PR[['BillWBS1', 'BillWBS2', 'BillWBS3', 'Address2', 'Address3', 'Zip', 'County', 'BidDate',


In [45]:
pr_dropcols = ["Fee", "ReimbAllow", "ConsultFee", "BudOHRate", "Status", "RevType", "MultAmt", "Org",
               "UnitTable", "PctComp",  "LabPctComp", "ExpPctComp", "BillableWarning", "Memo",
               "BudgetedFlag", "XCharge", "XChargeMethod", "XChargeMult", "Closed", "ReadOnly",
               "DefaultEffortDriven", "DefaultTaskType", "VersionID", "FirmCostComment", "TotalCostComment",
               "OpportunityID", "ClientConfidential", "AvailableForCRM", "ReadyForApproval", "ReadyForProcessing",
               "Phone", "Fax", "EMail", "ProposalWBS1", "CostRateMeth", "CostRateTableNo", "PayRateMeth",
               "PayRateTableNo", "ProfServicesComplDate", "ProjectExchangeRate", "BillingExchangeRate",
               "RestrictChargeCompanies", "FeeBillingCurrency",  "ReimbAllowBillingCurrency", "BidDate",
               "ConsultFeeBillingCurrency", "RevUpsetLimits", "RevUpsetWBS2", "RevUpsetWBS3",
               "RevUpsetIncludeComp", "RevUpsetIncludeCons", "RevUpsetIncludeReimb", "PORMBRate", "POCNSRate",
               "TKCheckRPDate", "ICBillingLab", "ICBillingLabMethod", "ICBillingLabMult", "ICBillingExp",
               "ICBillingExpMethod", "ICBillingExpMult", "CreateUser",  "CreateDate", "ModUser", "ModDate",
               "RequireComments",  "TKCheckRPPlannedHrs", "BillByDefaultConsultants",  "BillByDefaultOtherExp",
               "BillByDefaultORTable", "PhoneFormat", "FaxFormat", "RevType2", "RevType3", "RevType4", "RevType5",
               "RevUpsetCategoryToAdjust", "FeeFunctionalCurrency", "ReimbAllowFunctionalCurrency",
               "ConsultFeeFunctionalCurrency", "RevenueMethod", "ICBillingLabTableNo", "ICBillingExpTableNo",
               "FeeDirLab", "FeeDirExp", "ReimbAllowExp", "ReimbAllowCons", "FeeDirLabBillingCurrency",
               "FeeDirExpBillingCurrency", "ReimbAllowExpBillingCurrency", "ReimbAllowConsBillingCurrency",
               "FeeDirLabFunctionalCurrency", "FeeDirExpFunctionalCurrency", "ReimbAllowExpFunctionalCurrency",
               "ReimbAllowConsFunctionalCurrency", "RevUpsetIncludeCompDirExp", "RevUpsetIncludeReimbCons",
               "AjeraSync", "FESurchargePct", "FESurcharge", "FEAddlExpensesPct", "FEAddlExpenses", "FEOtherPct",
               "FEOther", "AjeraSpentLabor", "AjeraSpentReimbursable", "AjeraSpentConsultant", "AjeraCostLabor",
               "AjeraCostReimbursable", "AjeraCostConsultant", "AjeraWIPLabor", "AjeraWIPReimbursable",
               "AjeraWIPConsultant", "AjeraBilledLabor", "AjeraBilledReimbursable", "AjeraBilledConsultant",
               "AjeraReceivedLabor", "AjeraReceivedReimbursable", "AjeraReceivedConsultant", "PIMID"]

In [46]:
# adding dtype specifications to avoid issues
error_column_indices = [60,62,63,71,72,73,74,79,87,88,110,111]
## BE SURE TO UPDATE BEFORE RUNNING

# Map the column indices to column names
column_names = df_PR.columns
error_column_names = [column_names[i] for i in error_column_indices]

In [47]:
# Now filter out the ones I plan to drop anyway
pr_dropcols = [col for col in error_column_names if col not in pr_dropcols]
pr_dropcols

['ClientAlias']

In [48]:
# not sure why ['ClientAlias'] is still mixed type.  Easy enough to fix though.
df_PR['ClientAlias'].fillna('', inplace=True)

In [49]:
df_PR['ClientAlias'] = df_PR['ClientAlias'].astype(str)

In [50]:
df_PR['ClientAlias'].dtypes

dtype('O')

In [53]:
iterable_PR = [row for row in df_PR.index]
for i in iterable_PR:
    if type(df_PR['ClientAlias'][i]) != str:
        print(type(df_PR['ClientAlias'][i]))

In [54]:
## well I don't knwo what's up, every individual row is a str but it still has dtype('0')
# going to hope it's not an issue.

In [55]:
# this was how I quickly checked dtypes
# df_PR['##COLTOCHECK##'].value_counts()
# df_PR['##COLTOCHECK##'].isna().sum()
# df_PR[df_PR['##COLTOCHECK##'].notnull()] # get some indexes
# type(df_PR['##COLTOCHECK##'].loc[#INDEX#])

# df_PR['BillingContactID'].value_counts()
# df_PR['BillingContactID'].isna().sum()
# df_PR['BillingContactID'].dtypes
#  'FirmCostComment',
#  'TotalCostComment',
#  'OpportunityID',
#  'Phone',
#  'Fax',
#  'EMail',
#  'ProposalWBS1',
#  'ProfServicesComplDate',
#  'RevUpsetWBS2',
#  'RevUpsetWBS3',
#  'PhoneFormat',
#  'FaxFormat'
# ['FirmCost', 'TotalProjectCost', 'BillingContactID']
# BillingContactID as string, the others are fine as is.

In [56]:
# found an issue with nulls that weren't actually nulls
# df_PR['WBS3'][df_PR['WBS3'] == ' '] = None

# in fact I may as well apply that to all columns
df_PR = df_PR.applymap(lambda x: None if x == ' ' else x)

In [57]:
df_PR.drop(axis=1, labels=pr_dropcols, inplace=True)

In [58]:
df_PR.dropna(axis=1, how='all', inplace=True)
# Check null counts
null_counts = df_PR.isna().sum()

In [59]:
null_counts[0:25]

WBS1                 0
WBS2             15673
WBS3             38878
Name                 0
ChargeType           0
SubLevel             0
Principal         2960
ProjMgr           4029
Supervisor       21769
ClientID            48
CLAddress          428
Fee                  0
ReimbAllow           0
ConsultFee           0
BudOHRate            0
Status               0
RevType              0
MultAmt              0
Org                  2
StartDate         9387
EndDate          79608
PctComp              0
LabPctComp           0
ExpPctComp           0
BillByDefault        0
dtype: int64

In [60]:
## don't remember what this was, doesn't appear to apply to new version
# df_PR.drop([76357, 77407, 77507], inplace=True)
# df_PR.loc[77407]

In [61]:
# df_PR['Biller'].value_counts() # practically no information and unlikely to be relevant
# df_PR['LineItemApproval'].value_counts() # same value on every line, useless
# df_PR['LineItemApprovalEK'].value_counts() # this one has info but it's just not relevant to the project

In [62]:
# df_PR.drop(axis=1, labels=["AwardType", "Duration", "ContractTypeGovCon", "MasterContract",
#                            "Biller", "PlanID", "BillingCurrencyCode", "Locale", "LineItemApproval",
#                            "LineItemApprovalEK", "BudgetSource", "BudgetLevel", "ConstComplDate",
#                            "ProjectCurrencyCode"], inplace=True)
## Old version.  Most of these columns no longer exist at this point anyway.

# df_PR.drop(axis=1, labels=["Biller", "LineItemApproval", "LineItemApprovalEK"], inplace=True)
## again, made unnecessary by other cleaning

In [63]:
df_PR.shape

(80907, 156)

In [64]:
def column_detail(df, colname):
    col_length = len(df[colname])
    nans_count = df[colname].isna().sum()
    samples = df[df[colname].notnull()].sample(6)
    data_types = df[colname].dtypes
    val_ct = df[colname].value_counts()
    print(f'Column is {col_length} long, has {nans_count} NaNs, and has dtype: {data_types}')
    print(f'Sample of column content: {samples}')
    print(f'Value Counts: {val_ct}')

In [65]:
column_detail(df_PR, 'SubLevel')

Column is 80907 long, has 0 NaNs, and has dtype: object
Sample of column content:                   WBS1 WBS2 WBS3                 Name ChargeType SubLevel  \
61256  R10.2020.000355  001  NaN    Physical Security          R        Y   
44809  R01.2008.005847  001  002          Office Time          R        N   
39190  R01.2004.004120  002  NaN  Additional Services          R        Y   
66094  R01.2014.008079  017  001       Okinawa, Japan          R        N   
32254  R01.2000.02327H  005  ZZZ             AR PHASE          R        N   
38479  R01.2004.004009  001  ZZZ             AR PHASE          R        N   

       Principal  ProjMgr  Supervisor                          ClientID  ...  \
61256      309.0    311.0       311.0  065D685401974FB0B666E6E68CECF59D  ...   
44809       62.0     52.0        52.0                           ROTONDA  ...   
39190       58.0     58.0         NaN                        PORTOVECCH  ...   
66094       88.0     88.0        75.0  392CB1E84E98497795C

In [67]:
## And that's it
# df_PR.to_pickle('Vision_Data/working_data/PR.pkl')

In [ ]:
df_Projects_BillingProjections = dfs[10]

In [ ]:
df_Projects_BillingProjections.isna().sum()

In [ ]:
df_Projects_BillingProjections.shape

In [ ]:
df_Projects_ProjectSetup = dfs[11]

In [ ]:
df_Projects_ProjectSetup.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_Projects_ProjectSetup.isna().sum()

In [ ]:
df_Projects_ProjectSetup.shape

### WIP files [RESUME HERE]

In [57]:
## not to be confused with "working files"
df_AR.to_pickle('Vision_Data/working_data/AR.pkl')
df_CL.to_pickle('Vision_Data/working_data/CL.pkl')
df_EMProjectAssoc.to_pickle('Vision_Data/working_data/EMProjectAssoc.pkl')
df_LD.to_pickle('Vision_Data/working_data/LD.pkl')
df_LedgerAP.to_pickle('Vision_Data/working_data/LedgerAP.pkl')
df_LedgerEX.to_pickle('Vision_Data/working_data/LedgerEX.pkl')
df_LedgerMisc.to_pickle('Vision_Data/working_data/LedgerMisc.pkl')
df_Opportunity.to_pickle('Vision_Data/working_data/Opportunity.pkl')
df_OpportunityCustomTabFields.to_pickle('Vision_Data/working_data/OpportunityCustomTabFields.pkl')
df_PR.to_pickle('Vision_Data/working_data/PR.pkl')

### Pickle (export to .pkl)

In [ ]:
# This section takes a long time to run, uncomment when needed.
for i, df in enumerate(dfs):
    working_filename = 'Vision\\PKL\\' + str(df_names[i]) + '.pkl'
    df.to_pickle(working_filename)

In [ ]:
# # # single pickle for minor corrections
# # df_CL.to_pickle('Vision_Data/PKL/CL.pkl')
# df_LedgerAP.to_pickle('Vision_Data/PKL/LedgerAP.pkl')
# df_LedgerEX.to_pickle('Vision_Data/PKL/LedgerEX.pkl')
# df_LedgerMisc.to_pickle('Vision_Data/PKL/LedgerMisc.pkl')
# df_PR.to_pickle('Vision_Data/PKL/PR.pkl')

### Re-Import from pickles
The below will read in all .pkl files in the directory as dataframes with a name equal to 'df_' + the filename.

In [ ]:
# Get a list of all files in the folder
all_files = os.listdir('Vision/PKL/')
# Filter the list to include only CSV files
pkl_files = [file for file in all_files if file.endswith('.pkl')]
# make a list of full paths
file_path_list = ['Vision/PKL/' + file for file in pkl_files]

# pretty sure this wasn't serving any purpose

# # Extract the filenames and full paths
# full_paths = [os.path.abspath(os.path.join('Vision/', file)) for file in pkl_files]
names = [name.strip('.pkl') for name in pkl_files]

# Pair names with file paths using zip
dflist = list(zip(names, file_path_list))

# Create dataframes from all with matching names
for name, df in dflist:
    exec(f"df_{name} = pd.read_pickle(df)")

In [ ]:
# # single import for minor corrections
# df_CL = pd.read_pickle('Vision/PKL/CL.pkl')

### Basic EDA

In [ ]:
start_date = pd.to_datetime('2001-01-01')
end_date = pd.to_datetime('2023-11-01')
plot_range = df_AR['InvoiceDate'][(df_AR['InvoiceDate'] > start_date) & (df_AR['InvoiceDate'] < end_date)]
num_bins = pd.date_range(start=start_date, end=end_date, freq='M')

# Create a histogram
plt.figure(figsize=(96, 10))
plt.hist(plot_range , bins=num_bins, color='blue', edgecolor='black', align='right')

# Set labels and title
plt.xlabel('Invoice Date')
plt.ylabel('# Invoices (monthly)')
plt.title('Distribution of Invoice Dates')

# Set x-axis ticks to number of bins & y-ticks to steps of ten for the range of the data
plt.xticks(num_bins, rotation=45, ha='left')
plt.yticks(np.arange(0, 381, step=10))
# Format x-axis as dates
plt.gcf().autofmt_xdate()
# Set x-axis limits to desired range
plt.xlim(start_date, end_date)

# Add horizontal grid lines
plt.grid(axis='y', linestyle='-', alpha=0.7)

# Show the plot
plt.show()

#### Chart: All Invoices, by Date
- Potentially useful to establish periodicity
- Want to refine to income by month
- Want to compare to # proposals perhaps

Problems with the above: doesn't seem to be aligning bins correctly with dates, may not be binning by month as intended, xticks should be appearing at the end of each month bin, start and end are blank for some reason

In [ ]:
# establishing overall range
plot_range.sort_values()

In [ ]:
df_AR['InvoiceDate'].max()

In [ ]:
df_LD.head()

In [ ]:
# make pd.series of all P## project codes, R## project codes, and OHD codes

ohd_codes = df_LD[df_LD['WBS1'].str.startswith('OHD')]['WBS1'].value_counts()
# 372095 OHD codes
r_codes = df_LD[df_LD['WBS1'].str.startswith('R')]['WBS1'].value_counts()
# 696996 R codes
p_codes = df_LD[df_LD['WBS1'].str.startswith('P')]['WBS1'].value_counts()
# 272768 P codes

In [ ]:
r_codes

In [ ]:
# since NPS is a major client we'll use those for the test example
NPS_projects = df_PR[df_PR['ClientID'].isin(["NPS"])]

In [ ]:
# this function (WIP, PROP version still needs fixing) totals labor hours by client
# it matches all WBS1 codes associated with the ClientID in the PR table
# then finds all entries in LD associated with that WBS1
# it can also take parameters to only find hours marked "N" for NonBill,
# and (will) match only WBS2 = PROP hours 
# (I think this last needs to do some more cross-table magic to work properly)
def get_totals(filter_value, df_PR, df_LD, mode='default'):
    # Filter rows where 'ClientID' matches the filter value
    filtered_df = df_PR[df_PR['ClientID'].isin([filter_value])]

    # Get unique values from the 'WBS1' column in the filtered DataFrame
    unique_values = filtered_df['WBS1'].unique()

    # Convert the unique values to a list
    unique_values_list = list(unique_values)

    # Initialize dictionaries to store totals
    reg_hours_totals = {}
    ovt_hours_totals = {}

    # Apply additional filters based on mode
    if mode == 'proposal':
        df_LD_filtered = df_LD[df_LD['WBS2'].str.startswith('PROP')]
    elif mode == 'billable':
        df_LD_filtered = df_LD[df_LD['NonBill'].isin([None, 'N'])]
    else:
        df_LD_filtered = df_LD

    # Initialize a list to store the result strings
    result_strings = []

    # Iterate over unique values and calculate totals
    for wbs1_code in unique_values_list:
        relevant_rows = df_LD_filtered[df_LD_filtered['WBS1'] == wbs1_code]
        reg_hours_total = relevant_rows['RegHrs'].sum()
        ovt_hours_total = relevant_rows['OvtHrs'].sum()
        result_string = f"WBS1 Code: {wbs1_code}, Regular Hours: {reg_hours_total}, Overtime Hours: {ovt_hours_total}"
        result_strings.append(result_string)

    return result_strings

In [ ]:
# Example usage
result_strings = get_totals('NPS', df_PR, df_LD, mode='billable')
for result_string in result_strings:
    print(result_string)

In [ ]:
#  SAMPLE CODE - make sure none are repeated in a list
# TEMP_clientIDcounts = df_CL['ClientID'].value_counts()
# TEMP_clientIDcounts[TEMP_clientIDcounts > 1]